In [174]:
# import packages that needed for working with the data
import geopandas as gpd
import pandas as pd
import numpy as np

# DATA PREPARATION


In [175]:
# read and load covid_19_indonesia_time_series_all dataframe
df = pd.read_csv('/content/drive/MyDrive/Dataset/covid_19_indonesia_time_series_all.csv')
df.head()

,Date,Location ISO Code,Location,New Cases,New Deaths,New Recovered,New Active Cases,Total Cases,Total Deaths,Total Recovered,Total Active Cases,Location Level,City or Regency,Province,Country,Continent,Island,Time Zone,Special Status,Total Regencies,Total Cities,Total Districts,Total Urban Villages,Total Rural Villages,Area (km2),Population,Population Density,Longitude,Latitude,New Cases per Million,Total Cases per Million,New Deaths per Million,Total Deaths per Million,Case Fatality Rate,Case Recovered Rate,Growth Factor of New Cases,Growth Factor of New Deaths,Name,Item,Kind,Hidden
0,1/8/2020,ID-JI,Jawa Timur,9,3,1,5,9,5,10,-6,Province,NaN,Jawa Timur,Indonesia,Asia,Jawa,UTC+07:00,NaN,29,9.0,666,777.0,7724.0,47803,40479023,846.78,112.732941,-7.723346,0.22,0.22,0.07,0.12,55.56%,111.11%,NaN,NaN,NaN,NaN,NaN,NaN
1,1/9/2020,ID-JI,Jawa Timur,0,1,23,-24,9,6,33,-30,Province,NaN,Jawa Timur,Indonesia,Asia,Jawa,UTC+07:00,NaN,29,9.0,666,777.0,7724.0,47803,40479023,846.78,112.732941,-7.723346,0.00,0.22,0.02,0.15,66.67%,366.67%,0.0,0.33,NaN,NaN,NaN,NaN
2,1/10/2020,ID-JI,Jawa Timur,0,1,14,-15,9,7,47,-45,Province,NaN,Jawa Timur,Indonesia,Asia,Jawa,UTC+07:00,NaN,29,9.0,666,777.0,7724.0,47803,40479023,846.78,112.732941,-7.723346,0.00,0.22,0.02,0.17,77.78%,522.22%,1.0,1.00,NaN,NaN,NaN,NaN
3,1/11/2020,ID-JI,Jawa Timur,0,3,8,-11,9,10,55,-56,Province,NaN,Jawa Timur,Indonesia,Asia,Jawa,UTC+07:00,NaN,29,9.0,666,777.0,7724.0,47803,40479023,846.78,112.732941,-7.723346,0.00,0.22,0.07,0.25,111.11%,611.11%,1.0,3.00,NaN,NaN,NaN,NaN
4,1/12/2020,ID-JI,Jawa Timur,0,3,1,-4,9,13,56,-60,Province,NaN,Jawa Timur,Indonesia,Asia,Jawa,UTC+07:00,NaN,29,9.0,666,777.0,7724.0,47803,40479023,846.78,112.732941,-7.723346,0.00,0.22,0.07,0.32,144.44%,622.22%,1.0,1.00,NaN,NaN,NaN,NaN


In [176]:
# selecting the columns to integrate with geojson data
# New Active Cases = New Cases - (New Recovered + New Deaths)
df = df.groupby(['Location','Longitude', 'Latitude'])[['New Cases','New Recovered', 'New Deaths','New Active Cases']].sum().reset_index()
df = df.loc[df['Location'] != 'Indonesia']
df = df.reset_index(drop=True)
df.head()

,Location,Longitude,Latitude,New Cases,New Recovered,New Deaths,New Active Cases
0,Aceh,96.910522,4.225615,20140,15534,862,3744
1,Bali,115.131714,-8.369472,54079,46180,1618,6281
2,Bangka-Belitung,106.549932,-2.447444,23188,20732,365,2091
3,Banten,106.109004,-6.456736,61689,50392,1452,9845
4,Bengkulu,102.338421,-3.533584,11671,9530,194,1947


In [177]:
# read and load geojson data
geojson = gpd.read_file('/content/drive/MyDrive/Dataset/indonesia-edit.geojson')
geojson.head()

,cartodb_id,country,id_1,state,name,key,slug,geometry
0,16,Indonesia,1,Aceh,Special Region of Aceh,indonesia-aceh,special-region-of-aceh,"MULTIPOLYGON (((97.97681 4.62750, 97.99611 4.6..."
1,2,Indonesia,31,Sumatera Barat,West Sumatera,indonesia-sumaterabarat,west-sumatera,"MULTIPOLYGON (((99.17167 -1.50250, 99.11528 -1..."
2,8,Indonesia,34,Daerah Istimewa Yogyakarta,Special Region of Yogyakarta,indonesia-yogyakarta,special-region-of-yogyakarta,"MULTIPOLYGON (((110.70204 -8.18505, 110.83630 ..."
3,20,Indonesia,33,Sumatera Utara,North Sumatera,indonesia-sumaterautara,north-sumatera,"MULTIPOLYGON (((98.71384 3.76947, 98.72339 3.7..."
4,7,Indonesia,3,Bangka-Belitung,Bangka Belitung Islands,indonesia-bangkabelitung,bangka-belitung-islands,"MULTIPOLYGON (((105.34754 -1.84469, 105.12500 ..."


In [178]:
# create the function to extract New Cases, New Recovered,	New Deaths,	New Active Cases, Longitude, Latitude
# from covid_19_indonesia_time_series_all dataframe to geojson data

def get_total_cases(col):
    s = df['Location']
    if col in list(s):
        i = list(s).index(col)
        return df.loc[i, 'New Cases']
    else:
        return 0
    
def get_total_recovered(col):
    s = df['Location']
    if col in list(s):
        i = list(s).index(col)
        return df.loc[i, 'New Recovered']
    else:
        return 0
    
def get_total_deaths(col):
    s = df['Location']
    if col in list(s):
        i = list(s).index(col)
        return df.loc[i, 'New Deaths']
    else:
        return 0

def get_total_active_cases(col):
    s = df['Location']
    if col in list(s):
        i = list(s).index(col)
        return df.loc[i, 'New Active Cases']
    else:
        return 0
    
def get_latitude(col):
    s = df['Location']
    if col in list(s):
        i = list(s).index(col)
        return df.loc[i, 'Latitude']
    else:
        return 0.8035

def get_longitude(col):
    s = df['Location']
    if col in list(s):
        i = list(s).index(col)
        return df.loc[i, 'Longitude']
    else:
        return 122.0568

In [179]:
# create new columns to geojson data through the function that we've created before

geojson['New Cases'] = geojson['state'].apply(get_total_cases)
geojson['New Recovered'] = geojson['state'].apply(get_total_recovered)
geojson['New Deaths'] = geojson['state'].apply(get_total_deaths)
geojson['New Active Cases'] = geojson['state'].apply(get_total_active_cases)
geojson['Latitude'] = geojson['state'].apply(get_latitude)
geojson['Longitude'] = geojson['state'].apply(get_longitude)
geojson = geojson.rename(columns={'state':'Province'})
geojson.head()

,cartodb_id,country,id_1,Province,name,key,slug,geometry,New Cases,New Recovered,New Deaths,New Active Cases,Latitude,Longitude
0,16,Indonesia,1,Aceh,Special Region of Aceh,indonesia-aceh,special-region-of-aceh,"MULTIPOLYGON (((97.97681 4.62750, 97.99611 4.6...",20140,15534,862,3744,4.225615,96.910522
1,2,Indonesia,31,Sumatera Barat,West Sumatera,indonesia-sumaterabarat,west-sumatera,"MULTIPOLYGON (((99.17167 -1.50250, 99.11528 -1...",55675,48816,1259,5600,-0.850253,100.465062
2,8,Indonesia,34,Daerah Istimewa Yogyakarta,Special Region of Yogyakarta,indonesia-yogyakarta,special-region-of-yogyakarta,"MULTIPOLYGON (((110.70204 -8.18505, 110.83630 ...",72560,54220,1892,16448,-7.894502,110.444878
3,20,Indonesia,33,Sumatera Utara,North Sumatera,indonesia-sumaterautara,north-sumatera,"MULTIPOLYGON (((98.71384 3.76947, 98.72339 3.7...",38040,33773,1231,3036,2.191894,99.051964
4,7,Indonesia,3,Bangka-Belitung,Bangka Belitung Islands,indonesia-bangkabelitung,bangka-belitung-islands,"MULTIPOLYGON (((105.34754 -1.84469, 105.12500 ...",23188,20732,365,2091,-2.447444,106.549932


# GENERATE CHOROPLETH MAP

In [180]:
# import package for generating map
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

/bin/bash: conda: command not found
Folium installed and imported!


In [181]:
# initialize a map using folium centered with Indonesia and save it into indo_map object
indo_map = folium.Map(location=[0.7893, 113.9213], zoom_start=5, tiles='Mapbox Bright')

In [ ]:
# generate choropleth map using the new active cases of each province from January 2020 to July 2021
threshold_scale = np.linspace(geojson['New Active Cases'].min(),
                              geojson['New Active Cases'].max(),
                              8, dtype=int)
threshold_scale = threshold_scale.tolist()
threshold_scale[-1] = threshold_scale[-1] + 1

add_choropleth = folium.Choropleth(
    geo_data=geojson,
    data=geojson,
    columns=['Province', 'New Active Cases'],
    key_on='feature.properties.Province',
    threshold_scale=threshold_scale,
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='New Active Cases'
).add_to(indo_map)

#add the name of province on the map 
for i in range(0, len(geojson['Province'])):    
    temp = geojson.iloc[i]['Province']
    s = "Info"
    folium.Marker([geojson.iloc[i]['Latitude'], geojson.iloc[i]['Longitude']], icon = folium.DivIcon(html = f"""<div style = "font-family: fantasy
                    ; color: black; font-size: smaller; font-weight: boldest"> 
                    {"{}".format(temp) }</div>  """)).add_to(indo_map)

# add extra information about the number of new cases, new recovered, new deaths
add_choropleth.geojson.add_child(folium.features.GeoJsonTooltip(['Province','New Cases', 'New Recovered', 'New Deaths', 'New Active Cases']))

# display map
indo_map

In [183]:
# save to html
indo_map.save('/content/map.html')